<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Templates/tg/tg_bot_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TG bot template

## Install libs

In [1]:
%%capture
!pip install python-telegram-bot telebot aiogram

#### Install opencv

In [2]:
#for opencv
%%capture
!pip install opencv-python-headless

In [3]:
import cv2

## Imports

In [4]:
import asyncio
import logging
import io
import numpy as np
import os
import telebot


from aiogram import Bot, Dispatcher, types
from aiogram.filters.command import Command
from google.colab import userdata
from PIL import Image

## Logging

In [5]:
# Set up logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [6]:
user_history = {}

In [7]:
def log_user(message, photopath, result):
  user_id = message.from_user.id
  if user_id not in user_history:
      user_history[user_id] = []

  user_history[user_id].append({
        'message_id': message.message_id,
        'text': message.text,
        'photo_path': photopath,
        'result': result})

## NN logic

In [8]:
def nn_logic(img): #substitute for real nn logic
    img_cv = np.array(img)
    img_cv = cv2.cvtColor(img_cv, cv2.COLOR_RGB2BGR)

    gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        cv2.rectangle(img_cv, (x, y), (x + w, y + h), (0, 255, 0), 2)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img_cv, 'face', (x, y - 10), font, 0.9, (0, 255, 0), 2, cv2.LINE_AA)
    img_pil = Image.fromarray(cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB))

    return img_pil


## Bot logics

### Create bot

In [10]:
TOKEN = userdata.get('trybot')
bot = telebot.TeleBot(TOKEN)

### Bot message handlers for photos

In [12]:
@bot.message_handler(commands=['start'])
def handle_start(message):
    bot.send_message(message.chat.id, "Welcome! Send me a photo, and I'll process it.")

@bot.message_handler(commands=['help'])
def handle_help(message):
    # Provide help information
    help_message = (
        "This bot can only process photos. Here are the available commands:\n"
        "/start - Start the bot\n"
        "/help - Display this help message\n"
        "Send me a photo, and I'll process it!"
    )
    bot.send_message(message.chat.id, help_message)

@bot.message_handler(func=lambda message: True, content_types=['text'])
def handle_text(message):
    # Respond to other text messages
    bot.send_message(message.chat.id, "I can only process photos. Send me a photo to get started.")


### Bot photos handler

In [13]:
@bot.message_handler(content_types=['photo'])
def handle_photo(message):
  try:
    photo_file = bot.get_file(message.photo[-1].file_id)
    photo_path = bot.download_file(photo_file.file_path)
    orig_photo, processed_photo_path = process_photo(photo_path)
    log_user(message, orig_photo, processed_photo_path)
    send_photo(message.chat.id, processed_photo_path)
    logger.info(f"Photo processed for user {message.from_user.id}")
  except Exception as e:
    logger.error(f"Error processing photo: {str(e)}")

def process_photo(photo_path):
  orig_photo = 'input_photo.jpg'
  processed_photo_path = 'temp.jpg'
  photo = Image.open(io.BytesIO(photo_path))
  photo.save(orig_photo)

  processed_photo = nn_logic(photo)
  processed_photo.save(processed_photo_path)

  return orig_photo, processed_photo_path

def send_photo(chat_id, photo):
  bot.send_photo(chat_id, open(photo, 'rb'))
  os.remove(photo)

## Run bot

In [15]:
if __name__ == '__main__':
  bot.polling()

In [14]:
user_history

{}

#Async approach